In [3]:
print('hello')

hello


In [9]:
import surprise
from collections import defaultdict

from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
import pandas as pd
import requests
import re

In [56]:
k = 4
films_count = 5
id = input('Enter user ID: ')

data = surprise.Dataset.load_builtin('ml-100k')

trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': True, 'min_support': films_count}
algo = surprise.KNNBaseline(k=k, sim_options=sim_options)
algo.fit(trainset)

testset = trainset.build_anti_testset()
testset = filter(lambda x: x[0] == id, testset)

predictions = algo.test(testset)

top_n = defaultdict(list)
for uid, iid, _, est, _ in predictions:
    top_n[uid].append((iid, round(est, 3)))

for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:5]

file_name = surprise.get_dataset_dir() + '/ml-100k/ml-100k/u.item'
item = {}

with open(file_name, 'r') as f:
    for line in f:
        line = line.split('|')
        item[line[0]] = (line[1], line[2])

print(f'User {id}:')

movies = []
for movie_id, rating in top_n[id]:
    film = re.sub(r'[\s]{1}\([0-9]{4}\)', "", str(item[movie_id][0]))
    movies.append(film)
#     print(str(movie_id) + "\t" + str(rating) + "\t" + str(item[movie_id]))

movies

Enter user ID: 1
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
User 1:


['L.A. Confidential',
 'Santa with Muscles',
 'Great Day in Harlem, A',
 'Aiqing wansui',
 "Someone Else's America"]

In [87]:
def Sparql(sparql, film):
    
    queryString = """

    SELECT DISTINCT ?person ?personLabel ?film ?filmLabel 
    WHERE {
      wd:"""+ film +""" wdt:P57 ?person.
      ?film wdt:P31 wd:Q11424.
      ?film wdt:P57 ?director
      FILTER(?director = ?person).
      SERVICE wikibase:label { bd:serviceParam wikibase:language "ru,en". }
    }
    """
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.io.json.json_normalize(results['results']['bindings'])
    return results_df

In [88]:
for movie in movies:
    display(HTML('<h3>'+ movie + ' :</h3>'))
    params = {'action' : 'wbsearchentities','format' : 'json','language' : 'en','search': movie}
    res = requests.get(API_ENDPOINT, params = params)
    try:
        film = res.json()['search'][0]['id']
        print(film)
        res = Sparql(sparql, film);
        res = res.reindex(columns=['personLabel.value', 'filmLabel.value'])
        print(res)
  
    except:
        display(HTML('<p>film not found ¯\_(ツ)_/¯</p>'))

Q339876
   personLabel.value          filmLabel.value
0      Кёртис Хэнсон               Sweet Kill
1      Кёртис Хэнсон       The Little Dragons
2      Кёртис Хэнсон           Дурное влияние
3      Кёртис Хэнсон         Подальше от тебя
4      Кёртис Хэнсон               Дикая река
5      Кёртис Хэнсон                Losin' It
6      Кёртис Хэнсон    Секреты Лос-Анджелеса
7      Кёртис Хэнсон       The Bedroom Window
8      Кёртис Хэнсон              Вундеркинды
9      Кёртис Хэнсон             Восьмая миля
10     Кёртис Хэнсон          Покорители волн
11     Кёртис Хэнсон  Рука, качающая колыбель
12     Кёртис Хэнсон                 Везунчик


Q1631700
  personLabel.value                filmLabel.value
0    John Murlowski                   The Bad Twin
1    John Murlowski                 Freeway Killer
2    John Murlowski                Чёрный кадиллак
3    John Murlowski                 Terminal Error
4    John Murlowski                        Cop Dog
5    John Murlowski             Богатенький Ричи 2
6    John Murlowski  Амитивилль 7: Новое поколение
7    John Murlowski                      Q30766080
8    John Murlowski              Силач Санта-Клаус
9    John Murlowski          The Secret Agent Club


Q7219297
    personLabel.value            filmLabel.value
0   Паскалевич, Горан             Cabaret Balkan
1   Паскалевич, Горан  The Elusive Summer of '68
2   Паскалевич, Горан             Bitter Harvest
3   Паскалевич, Горан              The Optimists
4   Паскалевич, Горан             Guardian Angel
5   Паскалевич, Горан              Twilight Time
6   Паскалевич, Горан            Tango Argentino
7   Паскалевич, Горан            When Day Breaks
8   Паскалевич, Горан   The Dog Who Loved Trains
9   Паскалевич, Горан      Beach Guard in Winter
10  Паскалевич, Горан     Someone Else's America
11  Паскалевич, Горан    Midwinter Night's Dream
12  Паскалевич, Горан           Time of Miracles
13  Паскалевич, Горан          Special Treatment
14  Паскалевич, Горан                 Honeymoons
